In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_simulation_causalklgp as file

In [2]:
# Args setup
ntrial = 50
n = 500
n_int = 100
niter = 1000
front_door = True
minimise = False
calibrate = False
sample_split = True
add_base_kernel_BO = True

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
        f = client.submit(file.main,
                              seed,
                              n,n_int,
                              niter = niter,
                              front_door = front_door,
                              minimise = minimise,
                              calibrate = calibrate,
                              sample_split = sample_split,
                              add_base_kernel_BO = add_base_kernel_BO
                             )
        futures += [f]

In [5]:
futures

[<Future: pending, key: main-20a85e25375e347dddbba8eeeef2114b>,
 <Future: pending, key: main-be46114150ff44ad4330605e432e7133>,
 <Future: pending, key: main-cec91da4347a6be3fabc44c29bf1b9bd>,
 <Future: pending, key: main-a09b77c68fda8e05d1c90a63393cf388>,
 <Future: pending, key: main-eae8b8cc9a3e1da64fc3b5e4fdec1eeb>,
 <Future: pending, key: main-8f9a1a3e6bfaef4b0e2b81f4ca442b75>,
 <Future: pending, key: main-7113caf5baa4a17391c5a7fc78f837e5>,
 <Future: pending, key: main-c931fda884fab4e0c173c7cadfd19e08>,
 <Future: pending, key: main-3ca0f007e8ee40962ff474df89de9ce8>,
 <Future: pending, key: main-7e59ffa06328cefee1c6e7ce50d010f4>,
 <Future: pending, key: main-7bae0618d72d3625bee64f3258098a92>,
 <Future: pending, key: main-77a4f7cad9c9423224f7aeaadd81741f>,
 <Future: pending, key: main-874b18a4c2c53e1cb54189424d4731f3>,
 <Future: pending, key: main-d45ed511a22f30a51017474e697cdfdf>,
 <Future: pending, key: main-44ab5d3d79fcb4c7ae0dc9409d2c8fb8>,
 <Future: pending, key: main-f7b9dd01fa8

In [ ]:
# Getting results
results = client.gather(futures)

In [ ]:
# Closing client
client.close()
cluster.close()

In [ ]:
# Saving results
torch.save(obj = results,
           f = "simulation_causalklgp_ntrial={0}_n={1}_frontdoor={2}_minimise={3}_addbase={4}.pt".format(ntrial,
                                                                                             n,
                                                                                             front_door,
                                                                                             minimise,
                                                                                             add_base_kernel_BO)
          )